In [1]:
import pandas as pd
import re
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side

In [2]:
input = pd.read_excel('./Template.xlsx', dtype=str)
input['OE'] = input['OE'].apply(lambda x:re.sub('\\([^)]*\\)','',x))

In [3]:
def fetch_sku_num_1(oe):
    list_oe = oe.split(';')
    length_oe = len(list_oe)
    oe_clean =  ''.join(re.findall(r'\d', list_oe[0]))
    if len(oe_clean) > 4:
        sku_num = oe_clean[-5:]
    else:
        zero_conut = 5-len(oe_clean)
        sku_num = zero_conut*'0' + oe_clean
    return sku_num

def sku_clean(sku):
    # 以"-FC、-C、-YC"结尾，去掉相应字符
    sku_ = re.sub('-YC$', '', sku)
    sku_ = re.sub('-FC$', '', sku_)
    sku_ = re.sub('-C$', '', sku_)
    # 以"DEB-、DE、USPA、US"开头，去掉相应字符
    sku_ = re.sub('^DEB-', '', sku_)
    sku_ = re.sub('^DE', '', sku_)
    sku_ = re.sub('^USPA', '', sku_)
    sku_ = re.sub('^US', '', sku_)
    return sku_

def fetch_sku_num_2(oe):
    list_oe = oe.split(';')
    length_oe = len(list_oe)
    oe_clean =  ''.join(re.findall(r'\d', list_oe[0]))
    if len(oe_clean) > 5:
        sku_num = oe_clean[-6:-1]
    elif length_oe > 1:
        oe_clean =  ''.join(re.findall(r'\d', list_oe[1]))
        if len(oe_clean) > 4:
            sku_num = oe_clean[-5:]
        else:
            zero_conut = 5-len(oe_clean)
            sku_num = zero_conut*'0' + oe_clean
    else:
        zero_conut = 5 - len(oe_clean) + 1
        sku_num = zero_conut*'0' + oe_clean[:-1]
    return sku_num

def delete(sku):
    if len(sku) < 5:
        return ''
    elif sku[-1] == '-':
        return sku[:-1]
    else:
        return sku

In [4]:
df_sku_num = input.copy()
df_sku_num['middle'] = df_sku_num['OE'].apply(lambda x:fetch_sku_num_1(x))
df_sku_num['中性sku'] = df_sku_num['品类简称'] + df_sku_num['middle']
# df_sku_num

In [5]:
ptsku = pd.read_excel('./Template.xlsx', sheet_name='PTSKU', dtype=str)
ptsku_ = ptsku.copy()
ptsku_['SKU'] = ptsku_['SKU'].apply(lambda x: sku_clean(x))
ptsku_concat = pd.concat([ptsku, ptsku_])
ptsku_concat.drop_duplicates(subset=['SKU'], keep='last', inplace=True)
# ptsku_concat

In [6]:
df_sku_num_check = pd.merge(df_sku_num, ptsku_concat, left_on='中性sku', right_on='SKU',  how='left').fillna('')
df_ok = df_sku_num_check[df_sku_num_check['SKU']=='']
df_again = df_sku_num_check[df_sku_num_check['SKU']!='']

In [7]:
df_again = df_again[['No.', '品类简称', 'OE', '后缀']].copy()
df_again['middle'] = df_again['OE'].apply(lambda x: fetch_sku_num_2(x))
df_again['中性sku'] = df_again['品类简称'] + df_again['middle']

In [8]:
df_sku_num_check_2 = pd.merge(df_again, ptsku_concat, left_on='中性sku', right_on='SKU',  how='left').fillna('')
df_ok_2 = df_sku_num_check_2[df_sku_num_check_2['SKU']=='']
df_ok_ = pd.concat([df_ok, df_ok_2])
# df_ok_

In [9]:
df_result_sku = pd.merge(input, df_ok_[['No.', '中性sku']], on='No.', how='left').fillna('')
df_result_sku['sku命名'] = df_result_sku['中性sku'] + '-' + df_result_sku['后缀']
df_result_sku['sku命名'] = df_result_sku['sku命名'].apply(lambda x: delete(x))
# df_result_sku

In [10]:
def fetch_vehicle(des):
    try:
        list_des = des.split(' ')
        year_alone = [str(i) for i in range(1950, 2050)]
        for i in list_des:
            if '-' in i:
                if len(i.split('-')[0]) == 4 and len(i.split('-')[-1]) == 4:
                    index_g = list_des.index(i)
                    before_g = list_des[:index_g]
                    for j in before_g:
                        if j in year_alone:
                            index_g = before_g.index(j)
                            break
                    break
            else:
                for k in list_des:
                    if k in year_alone:
                        index_g = list_des.index(k)
                        break
        vehicle = ' '.join(list_des[1:index_g])
        return vehicle
    except:
        return ''
    
df_result_sku['Vehicle'] = df_result_sku['Description'].apply(lambda x:fetch_vehicle(x))
df_result_sku['Vehicle_'] = df_result_sku['Vehicle'].apply(lambda x:x.title())
df_vehicle_cn = pd.read_excel('./Template.xlsx', sheet_name='中文车型翻译库（随时更新）', dtype=str)
df_vehicle_cn['英文车型_'] = df_vehicle_cn['英文车型'].apply(lambda x:x.title())
df_result = pd.merge(df_result_sku, df_vehicle_cn, left_on='Vehicle_', right_on='英文车型_', how='left').fillna('')
df_result = df_result[['No.', '品类简称', '后缀', 'OE', 'Description', 'Vehicle', '中性sku', 'sku命名', '中文车型']]
df_result['No.'] = df_result['No.'].apply(lambda x: int(x))
df_result

,No.,品类简称,后缀,OE,Description,Vehicle,中性sku,sku命名,中文车型
0,1,RCFK,C,12567;13996;15456;21040;20408;12757;20359;1172...,For Buick Century 1997-2005 Clip Retainer Fast...,Buick Century,RCFK12567,RCFK12567-C,别克世纪
1,2,RCFK,C,22527;11146;15414;12567;13996;22100;20988;CL10...,For Buick Cascada 2016-2019 Clip Retainer Fast...,Buick Cascada,RCFK22527,RCFK22527-C,别克卡斯卡达
2,3,RCFK,C,20891;12567;13996;22100;20988;CL10297;15456;21...,For Buick Encore 2013-2022 Clip Retainer Faste...,Buick Encore,RCFK20891,RCFK20891-C,别克昂科拉
3,4,RCFK,C,18301;cl101-145;11721;21365;21488;14921;22240;...,For Buick Envision 2016-2020 Clip Retainer Fas...,Buick Envision,RCFK18301,RCFK18301-C,别克昂科威
4,5,RCFK,C,17664;12567;13996;22100;20988;CL10297;15456;21...,For Buick Envision 2021-2024 Clip Retainer Fas...,Buick Envision,RCFK17664,RCFK17664-C,别克昂科威
...,...,...,...,...,...,...,...,...,...
155,156,RCFK,C,11719;8435;11146;15414;17664;15480;12757;20359...,For Saturn Aura 2007-2009 Clip Retainer Fasten...,Saturn Aura,RCFK11719,RCFK11719-C,
156,157,RCFK,C,18030;20359;20281;17975;CL10012;22527;11146;15...,For Saturn L series 2000-2005 Clip Retainer Fa...,Saturn L series,RCFK18030,RCFK18030-C,
157,158,RCFK,C,21374;20482;11146;15414;17664;15480;19270;1929...,For Saturn Relay 2005-2007 Clip Retainer Faste...,Saturn Relay,RCFK21374,RCFK21374-C,
158,159,RCFK,C,11146;15414;17664;12567;13996;17216;14987;2099...,For Saturn Outlook 2007-2010 Clip Retainer Fas...,Saturn Outlook,RCFK11146,RCFK11146-C,


In [11]:
wb = Workbook()
ws = wb.active
for r_idx, row in enumerate(dataframe_to_rows(df_result, index=False, header=True), 1):
    ws.append(row)
ws.column_dimensions['A'].width = 8
ws.column_dimensions['B'].width = 10
ws.column_dimensions['C'].width = 8
ws.column_dimensions['D'].width = 25
ws.column_dimensions['E'].width = 25
ws.column_dimensions['F'].width = 20
ws.column_dimensions['G'].width = 20
ws.column_dimensions['H'].width = 20
ws.column_dimensions['I'].width = 20
ws.freeze_panes = "A2"
font = Font(name='微软雅黑', size=9)
header_font = Font(name='微软雅黑', size=9, bold=True, color='FFFFFF')
fill = PatternFill(start_color="95B3D7", end_color="95B3D7", fill_type="solid")
alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)
for row in ws.iter_rows(min_row=2, max_col=ws.max_column, max_row=ws.max_row):
    for cell in row:
        cell.font = font
        cell.alignment = alignment
        cell.border = border
for cell in ws[1]:
    cell.font = header_font
    cell.fill = fill
    cell.alignment = alignment
    cell.border = border
ws['H1'].fill = PatternFill(start_color="92D050", end_color="92D050", fill_type="solid")
ws['I1'].fill = PatternFill(start_color="92D050", end_color="92D050", fill_type="solid")
for row in range(1, ws.max_row + 1):
    ws.row_dimensions[row].height = 20
wb.save('Output.xlsx')